In [27]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch import nn, optim
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.autograd import Variable
import math

In [45]:
X = torch.tensor(np.arange(0, 31), dtype=torch.float32).view(-1, 1)
Y = torch.tensor(np.array([30, 35, 33, 32, 34, 37, 39, 38, 36, 36, 37, 39, 42, 45, 45, 41, 40, 39, 42, 44, 47, 49, 50, 49, 46, 48, 50, 53, 55, 54, 53]), dtype=torch.float32).view(-1, 1)


In [44]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(1, 32)
        self.layer2 = nn.Linear(32, 64)
        self.layer3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.layer3(x)
        return x


In [41]:
def train_and_test_nn(train_indices, test_indices, epochs):
    model = NeuralNetwork()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.MSELoss()

    train_X = X[train_indices]
    train_Y = Y[train_indices]
    test_X = X[test_indices]
    test_Y = Y[test_indices]

    num_epochs = epochs
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        output = model(train_X)
        loss = criterion(output, train_Y)
        loss.backward()
        optimizer.step()
        if (epoch % 100) == 0:
            print(f"Epoch {epoch} loss: {loss}")

    preds_train = model(train_X)
    train_loss = criterion(preds_train, train_Y)

    preds_test = model(test_X)
    test_loss = criterion(preds_test, test_Y)
    return train_loss, test_loss


In [48]:


print("Case (ii): First 20 points as training data")
train_error, test_error = train_and_test_nn(np.arange(20), np.arange(20, 30), 2000)
print(f"\nTrain Error: {train_error}\nTest Error: {test_error}")

print("\nCase (iii): First 10 and last 10 points as training data")
train_error2, test_error2 = train_and_test_nn(np.hstack([np.arange(10), np.arange(21, 31)]), np.arange(10, 21), 2000)
print(f"\nTrain Error: {train_error2}\nTest Error: {test_error2}")

Case (ii): First 20 points as training data
Epoch 0 loss: 1408.250244140625
Epoch 100 loss: 5.858303070068359
Epoch 200 loss: 5.080464839935303
Epoch 300 loss: 4.916697978973389
Epoch 400 loss: 4.752251625061035
Epoch 500 loss: 4.680600643157959
Epoch 600 loss: 4.656827926635742
Epoch 700 loss: 4.641770362854004
Epoch 800 loss: 4.622031211853027
Epoch 900 loss: 4.598869323730469
Epoch 1000 loss: 4.574982166290283
Epoch 1100 loss: 4.553575038909912
Epoch 1200 loss: 4.53945779800415
Epoch 1300 loss: 4.535706043243408
Epoch 1400 loss: 4.54232931137085
Epoch 1500 loss: 4.538817405700684
Epoch 1600 loss: 4.669854640960693
Epoch 1700 loss: 4.6919403076171875
Epoch 1800 loss: 4.530725955963135
Epoch 1900 loss: 4.530873775482178

Train Error: 4.737574577331543
Test Error: 6.215279579162598

Case (iii): First 10 and last 10 points as training data
Epoch 0 loss: 2047.760986328125
Epoch 100 loss: 3.8218345642089844
Epoch 200 loss: 3.124260425567627
Epoch 300 loss: 2.7281723022460938
Epoch 400 los

In [32]:
#HW 1 Scores:
#1
# Error for Line (Train): 2.242749386808538
# Error for Parabola (Train): 2.1255393482773766
# Error for Polynomial (Train): 0.028351503968806435
# Error for Line (Test): 3.36363873604787
# Error for Parabola (Test): 8.713651781874919
# Error for Polynomial (Test): 28617752784.428474

#2
# Error for Line (Train): 1.851669904329375
# Error for Parabola (Train): 1.8508364115957907
# Error for Polynomial (Train): 0.1638133765080727
# Error for Line (Test): 2.8065076975181618
# Error for Parabola (Test): 2.774982896893291
# Error for Polynomial (Test): 483.9099124568562

In [33]:
#HW1
X = np.arange(0, 31)
Y = np.array([30, 35, 33, 32, 34, 37, 39, 38, 36, 36, 37, 39, 42, 45, 45, 41,
40, 39, 42, 44, 47, 49, 50, 49, 46, 48, 50, 53, 55, 54, 53])
X_train = X[:20]
X_test = X[20:]
Y_train = Y[:20]
Y_test = Y[20:]
def calc_error(coeffs, X, Y):
    function = np.poly1d(coeffs)
    pred = function(X)
    errors2 = (pred - Y) ** 2
    return math.sqrt(np.sum(errors2) / len(X))
line = np.polyfit(X_train, Y_train, 1)
parabola = np.polyfit(X_train, Y_train, 2)
poly = np.polyfit(X_train, Y_train, 19)

error_line_train = calc_error(line, X_train, Y_train)
error_parabola_train = calc_error(parabola, X_train, Y_train)
error_poly_train = calc_error(poly, X_train, Y_train)
error_line_test = calc_error(line, X_test, Y_test)
error_parabola_test = calc_error(parabola, X_test, Y_test)
error_poly_test = calc_error(poly, X_test, Y_test)
error_vals = [error_line_train, error_parabola_train, error_poly_train,
              error_line_test, error_parabola_test, error_poly_test]
names2 = ["Line (Train)", "Parabola (Train)", "Polynomial (Train)",
          "Line (Test)", "Parabola (Test)", "Polynomial (Test)"]
for i in range(6):
    print(f"Error for {names2[i]}: {error_vals[i]}")

#Part II (iv)
X_train2 = X[:10]
X_train2 = np.concatenate((X_train2, X[21:]))
Y_train2 = Y[:10]
Y_train2 = np.concatenate((Y_train2, Y[21:]))
X_test2 = X[10:21]
Y_test2 = Y[10:21]
line2 = np.polyfit(X_train2, Y_train2, 1)
parabola2 = np.polyfit(X_train2, Y_train2, 2)
poly2 = np.polyfit(X_train2, Y_train2, 19)
error_line_train2 = calc_error(line2, X_train2, Y_train2)
error_parabola_train2 = calc_error(parabola2, X_train2, Y_train2)
error_poly_train2 = calc_error(poly2, X_train2, Y_train2)
error_line_test2 = calc_error(line2, X_test2, Y_test2)
error_parabola_test2 = calc_error(parabola2, X_test2, Y_test2)
error_poly_test2 = calc_error(poly2, X_test2, Y_test2)
error_vals = [error_line_train2, error_parabola_train2, error_poly_train2,
              error_line_test2, error_parabola_test2, error_poly_test2]
names3 = ["Line (Train)", "Parabola (Train)", "Polynomial (Train)",
          "Line (Test)", "Parabola (Test)", "Polynomial (Test)"]
for i in range(6):
    print(f"Error for {names2[i]}: {error_vals[i]}")



Error for Line (Train): 2.242749386808538
Error for Parabola (Train): 2.1255393482773766
Error for Polynomial (Train): 0.028351503968806435
Error for Line (Test): 3.36363873604787
Error for Parabola (Test): 8.713651781874919
Error for Polynomial (Test): 28617752784.428474
Error for Line (Train): 1.851669904329375
Error for Parabola (Train): 1.8508364115957907
Error for Polynomial (Train): 0.1638133765080727
Error for Line (Test): 2.8065076975181618
Error for Parabola (Test): 2.774982896893291
Error for Polynomial (Test): 483.9099124568562


C:\Users\jleus\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\jleus\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)


In [52]:
import torch
import torchvision
from torchvision import transforms
from sklearn.decomposition import PCA
import numpy as np

# Load the MNIST train and test datasets
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = torchvision.datasets.MNIST('~/.torch/datasets', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST('~/.torch/datasets', train=False, download=True, transform=transform)

# Compute the first 20 PCA modes of the digit images
train_data = train_dataset.data.numpy().reshape(-1, 28*28)
pca = PCA(n_components=20)
pca.fit(train_data)
print(f'First 20 PCA Modes:\n{pca.components_}')

# Loaders for the train and test datasets
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000)

# Define the neural network
class FeedForwardNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(28 * 28, 256)
        self.fc2 = torch.nn.Linear(256, 128)
        self.fc3 = torch.nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model, loss function, and optimizer
model = FeedForwardNN()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Train the neural network
num_epochs = 10
for epoch in range(num_epochs):
    for i, (data, targets) in enumerate(train_loader):
        # Reshape the data and perform the forward pass
        data = data.view(data.size(0), -1)
        predictions = model(data)

        # Compute the loss, gradients, and update the parameters
        loss = criterion(predictions, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')


# Test the neural network
correct = 0
total = 0
with torch.no_grad():
    for data, targets in test_loader:
        # Reshape the data and perform the forward pass
        data = data.view(data.size(0), -1)
        predictions = model(data)

        # Compute the number of correct predictions
        _, predicted = torch.max(predictions.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}')


First 20 PCA Modes:
[[ 5.58521229e-18 -4.33747086e-19  4.50231512e-19 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.08824866e-16 -1.56249716e-17  1.50361393e-17 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 2.05768397e-17  3.22957539e-17 -3.94120299e-17 ... -0.00000000e+00
  -0.00000000e+00 -0.00000000e+00]
 ...
 [ 5.15809189e-17  6.45184807e-17  2.06693207e-16 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 5.28105334e-17 -5.91629520e-17 -5.12231420e-17 ... -0.00000000e+00
  -0.00000000e+00 -0.00000000e+00]
 [-6.71233474e-17  5.03922476e-17  8.40276715e-17 ... -0.00000000e+00
  -0.00000000e+00 -0.00000000e+00]]
Epoch [1/10], Step [100/938], Loss: 1.9406
Epoch [1/10], Step [200/938], Loss: 1.2251
Epoch [1/10], Step [300/938], Loss: 0.8189
Epoch [1/10], Step [400/938], Loss: 0.6054
Epoch [1/10], Step [500/938], Loss: 0.3623
Epoch [1/10], Step [600/938], Loss: 0.4840
Epoch [1/10], Step [700/938], Loss: 0.4193
Epoch [1/10], Step [800/938], Loss: 0.3262



KeyboardInterrupt



In [35]:
#HW3 Scores:
#Tree: 0.8769
#SVM: 0.9792
#LDA: 0.8730


In [36]:
#HW3

#SVM LDA and Decision Trees
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
from keras.datasets import mnist
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC


In [37]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 784).T
X_test = X_test.reshape(10000, 784).T

In [38]:
tree = DecisionTreeClassifier(random_state=44)
tree.fit(X_train.T, y_train)
y_pred_tree = tree.predict(X_test.T)
accuracy_tree = accuracy_score(y_test, y_pred_tree)
print(accuracy_tree)

0.8769


In [39]:
svm = SVC(probability=False)
svm.fit(X_train.T, y_train)
y_pred_svm = svm.predict(X_test.T)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(accuracy_svm)

0.9792


In [53]:

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
import numpy as np

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = torchvision.datasets.MNIST('~/.torch/datasets', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST('~/.torch/datasets', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000)

class MnistLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(MnistLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda()

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = MnistLSTM(input_size, hidden_size, num_layers, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 28, 28).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

with torch.no_grad():
    correct = 0
    total = 0

    for images, labels in test_loader:
        images = images.reshape(-1, 28, 28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {:.2f}%'.format(100 * correct / total))


Epoch [1/10], Step [100/938], Loss: 0.4337
Epoch [1/10], Step [200/938], Loss: 0.2637
Epoch [1/10], Step [300/938], Loss: 0.1552
Epoch [1/10], Step [400/938], Loss: 0.2181
Epoch [1/10], Step [500/938], Loss: 0.2380
Epoch [1/10], Step [600/938], Loss: 0.1099
Epoch [1/10], Step [700/938], Loss: 0.0993
Epoch [1/10], Step [800/938], Loss: 0.1525
Epoch [1/10], Step [900/938], Loss: 0.1009
Epoch [2/10], Step [100/938], Loss: 0.0430
Epoch [2/10], Step [200/938], Loss: 0.2676
Epoch [2/10], Step [300/938], Loss: 0.2343
Epoch [2/10], Step [400/938], Loss: 0.4767
Epoch [2/10], Step [500/938], Loss: 0.1476
Epoch [2/10], Step [600/938], Loss: 0.0777
Epoch [2/10], Step [700/938], Loss: 0.0345
Epoch [2/10], Step [800/938], Loss: 0.0841
Epoch [2/10], Step [900/938], Loss: 0.0867
Epoch [3/10], Step [100/938], Loss: 0.0905
Epoch [3/10], Step [200/938], Loss: 0.1358
Epoch [3/10], Step [300/938], Loss: 0.0739
Epoch [3/10], Step [400/938], Loss: 0.0190
Epoch [3/10], Step [500/938], Loss: 0.0059
Epoch [3/10